In [1]:
import scipy as sp
import numpy as np
import sndatasets
import sncosmo
import matplotlib as plt

%matplotlib inline

In [2]:
"""
sne = sndatasets.fetch_hamuy96()

data = sne['1992ae']

print data
"""

"\nsne = sndatasets.fetch_hamuy96()\n\ndata = sne['1992ae']\n\nprint data\n"

In [3]:
"""
model = sncosmo.Model(source='salt2')
res, fitted_model = sncosmo.fit_lc(data, model, ['z', 't0', 'x0', 'x1', 'c'], bounds={'z':(0.3, 0.7)})

#OR if we already knew the redshift...
#model.set(z=0.5)  # set the model's redshift.
#res, fitted_model = sncosmo.fit_lc(data, model, ['t0', 'x0', 'x1', 'c'])
"""

"\nmodel = sncosmo.Model(source='salt2')\nres, fitted_model = sncosmo.fit_lc(data, model, ['z', 't0', 'x0', 'x1', 'c'], bounds={'z':(0.3, 0.7)})\n\n#OR if we already knew the redshift...\n#model.set(z=0.5)  # set the model's redshift.\n#res, fitted_model = sncosmo.fit_lc(data, model, ['t0', 'x0', 'x1', 'c'])\n"

In [4]:
"""
print res.keys()

#print res.ncall  # number of chi^2 function calls made

#print res.ndof  # number of degrees of freedom in fit

#print res.chisq  # chi^2 value at minimum

#print res.parameters
"""

'\nprint res.keys()\n\n#print res.ncall  # number of chi^2 function calls made\n\n#print res.ndof  # number of degrees of freedom in fit\n\n#print res.chisq  # chi^2 value at minimum\n\n#print res.parameters\n'

In [5]:
#sncosmo.plot_lc(data, model=fitted_model, errors=res.errors)

In [6]:
#PLAYING WITH DISTRIBUTIONS
#Each simulated dataset has 250 SNe, except the highest-redshift, which has 50.
size_we_want = 250

#We generate the x1 population from a unit normal distribution, centered on zero.
x1_true_dist = np.random.normal(loc=0, scale=1, size=size_we_want)

#We assume that the uncertainties on mB, x1, and c are 0.05, 0.5, and 0.05, and are uncorrelated.
x1_obs_unc = 0.5*np.ones_like(x1_true_dist)

x1_obs_dist = x1_true_dist + x1_obs_unc*np.random.normal(loc=0, scale=1, size=size_we_want)

#We draw the population c values from the sum of a Gaussian distribution of width 0.1 magnitudes
#and an exponential with rate 1/(0.1 magnitudes). We center the distribution on zero.
#\beta is the scale parameter, which is the inverse of the rate parameter \lambda = 1/\beta. 
#The rate parameter is an alternative, widely used parameterization of the exponential distribution [R193].
c_true_norm = np.random.normal(loc=0, scale=0.1, size=size_we_want)
c_true_exp = np.random.exponential(scale=0.1/1, size=size_we_want)
c_true_dist = np.add(c_true_norm, c_true_exp)

c_obs_unc = 0.5*np.ones_like(c_true_dist)

c_obs_dist = c_true_dist + c_obs_unc*np.random.normal(loc=0, scale=1, size=size_we_want)

#We generate four simulated datasets spanning the redshift ranges 0.02–0.05, 
#0.05–0.4, 0.2–1.0, and 0.7–1.4 
##We're just doing one set for now.
z_dist = np.random.uniform(0.2, 1.0, size=size_we_want)

#α and β are assumed to be constant, with values 0.13 and 3.0, respectively. MB is set to −19.1 and
#Ωm is set to 0.3 (flat ΛCDM model).
alpha = 0.13
beta = 3.0
MB = -19.1
Omega_m = 0.3
sigma_int = 0.15

#Calculating mu values from z and Omega_m
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=Omega_m)
mu_dist = cosmo.distmod(z_dist).value
#print mu_dist

#We assume that the intrinsic dispersion covariance matrix is correct in SALT2, and that only dispersion
#in mB (gray dispersion) remains. The statistical model does not have access to this information,
#and fits for the full unknown matrix, overestimating the uncertainties on x1 and c, and thus slightly
#biasing α and β away from zero (see Section 2.5). (This is not a unique problem for our framework;
#the old technique would have the same bias.)
##Come back to this?

mb_true_dist = MB - alpha*x1_true_dist + beta*c_true_dist + mu_dist + np.random.normal(loc=0, scale=sigma_int, size=size_we_want)
 
mb_obs_unc = 0.05*np.ones_like(mb_true_dist)

mb_obs_dist = mb_true_dist + mb_obs_unc*np.random.normal(loc=0, scale=1, size=size_we_want)

print mb_obs_dist

[ 23.91680823  24.08601115  24.2980827   24.50728404  21.22083608
  23.07273467  24.55520405  22.56099017  24.98374653  24.19451674
  26.3294709   23.38206158  24.30781651  22.78373423  24.14699253
  22.80891817  23.89313331  24.53624868  24.60868736  21.70179801
  24.08185628  24.13480007  22.9020263   22.78830136  24.5190155
  23.4936208   24.61772887  23.32310615  23.8622788   25.16065415
  25.12096633  25.53044801  25.37465384  21.42078951  24.559321
  23.20826137  24.64576326  22.99447113  24.0588088   23.05306831
  23.75997298  24.11216501  24.91431336  23.4029289   24.75639659
  24.65253614  24.19517874  22.53585397  24.94861324  24.92282491
  24.2159408   21.33026432  22.15307924  24.1019423   25.13449863
  24.49991087  21.45388259  23.40526448  25.03515138  23.72689469
  23.26071126  23.30292223  25.07196227  25.3899938   23.55513872
  23.84789749  23.41942247  22.76583012  23.34144294  25.31408342
  23.17190263  23.55086782  21.41215151  24.1029374   21.75682825
  21.57318556